Scraping [derby names](https://en.wikipedia.org/wiki/Roller_derby#Derby_names) from publicly-accessible lists

In [16]:
import sys
!{sys.executable} -m pip install -qq pandas requests bs4 lxml advertools html5lib tqdm

In [17]:
import string
import random
from datetime import datetime
from pathlib import Path

from bs4 import BeautifulSoup
import requests

import numpy as np
import pandas as pd
import advertools as adv
from tqdm import tqdm

# from tqdm.auto import tqdm


In [18]:
if "google.colab" in sys.modules:
    from google.colab import files
    from sklearn.utils import shuffle


In [19]:
session = requests.Session()


In [20]:
# Download the WFTDA list of certified officials
try:
    wftda_df = pd.DataFrame()
    url = "https://resources.wftda.org/officiating/roller-derby-certification-program-for-officials/roster-of-certified-officials/"
    print("Downloading names from {}".format(url))
    session.headers.update({"User-Agent": "Mozilla/5.0"})
    r = session.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    rows = soup.find_all("h5")
    urls = [r.find("a")["href"] for r in rows]
    names = [r.find("a").get_text() for r in rows]
    wftda_df = pd.DataFrame({"Name": names, "url": urls})
except Exception as e:
    print(e)


In [21]:
if len(wftda_df) == 0:
    wftda_df = pd.read_csv("wftda.csv")
else:
    wftda_df.to_csv("wftda.csv", index=False)
wftda_df


,Name,url
0,9mm Ram-Paige,https://resources.wftda.org/officiating/roller...
1,A. Grue,https://resources.wftda.org/officiating/roller...
2,A’Blazing Grace,https://resources.wftda.org/officiating/roller...
3,Adam Smasher,https://resources.wftda.org/officiating/roller...
4,Admiral Mayhem,https://resources.wftda.org/officiating/roller...
...,...,...
550,Yu Cypher,https://resources.wftda.org/officiating/roller...
551,Yvel Saint Laurent,https://resources.wftda.org/officiating/roller...
552,Zebra 3,https://resources.wftda.org/officiating/roller...
553,Zero,https://resources.wftda.org/officiating/roller...


In [22]:
# Download TwoEvils list of skaters
url = "https://www.twoevils.org/rollergirls/"
print("Downloading names from %s" % url)
twoevils_df = pd.DataFrame()

try:
    twoevils_df = pd.read_html(url, skiprows=1)[0]
    twoevils_df.columns = [h.replace("Skater", "").strip() for h in twoevils_df.iloc[0]]
    twoevils_df = twoevils_df.rename(columns={"Date Added": "Registered"})
    twoevils_df = twoevils_df.iloc[1:-1, :].dropna(how="all")
    twoevils_df["url"] = url
except Exception as e:
    print(e)


<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>


In [23]:
if len(twoevils_df) == 0:
    twoevils_df = pd.read_csv("twoevils.csv")
else:
    twoevils_df.to_csv("twoevils.csv", index=False)
twoevils_df


,Name,Number,Registered,League,url
0,!(ED,REF,2012-02-20,Santiago Roller Derby,https://www.twoevils.org/rollergirls/
1,!Wolfespit,16,2011-02-01,Chilli Padi Derby Grrrls,https://www.twoevils.org/rollergirls/
2,"""A"" Cup Annihilator",36A,2010-10-13,Billings Roller Derby,https://www.twoevils.org/rollergirls/
3,"""Bobby"" Val Halen",1984,2011-02-24,Roller Derby Quebec,https://www.twoevils.org/rollergirls/
4,"""Chupa'clark'bra""",333,2012-01-09,Killa Hurtz Roller Girls,https://www.twoevils.org/rollergirls/
...,...,...,...,...,...
40536,Zuul,REF,2008-04-13,Orange County Roller Girls,https://www.twoevils.org/rollergirls/
40537,Zuzi Power,33,2012-04-20,White Horse Warriors,https://www.twoevils.org/rollergirls/
40538,Zwen Garden,E13,2011-03-24,Deja Vu Roller Derby,https://www.twoevils.org/rollergirls/
40539,Zyklon C,HCN1,2010-03-08,Croydon Roller Derby,https://www.twoevils.org/rollergirls/


In [24]:
# Download list from Derby Roll Call
try:
    drc_df = pd.DataFrame()
    url = "http://www.derbyrollcall.com/everyone"
    print("Downloading names from %s" % url)
    drc_df = pd.concat(pd.read_html(url))
    drc_df = drc_df.rename(columns={"#": "Number"})
    drc_df["url"] = url
except Exception as e:
    print(e)


In [25]:
if len(drc_df) == 0:
    drc_df = pd.read_csv("derbyrollcall.csv")
else:
    drc_df.to_csv("derbyrollcall.csv", index=False)
drc_df


,Name,Number,League,Country,Registered,url
0,Sausage Roller,M14,Arcadia Roller Derby,United Kingdom,"1st January, 1970",http://www.derbyrollcall.com/everyone
1,James Mean,27,Manchester Roller Derby,United Kingdom,"20th January, 2014",http://www.derbyrollcall.com/everyone
2,Yvel Saint Laurent,14,Knights of Oldham Roller Derby & Rainy City Ro...,United Kingdom,"20th January, 2014",http://www.derbyrollcall.com/everyone
3,Pancake,1928,Rainy City Roller Derby,United Kingdom,"20th January, 2014",http://www.derbyrollcall.com/everyone
4,Robert Quadriguez,101,Crash Test Brummies,United Kingdom,"20th January, 2014",http://www.derbyrollcall.com/everyone
...,...,...,...,...,...,...
39192,Brawly Shore,711,Chicago-Style Roller Derby,United States,"6th December, 2022",http://www.derbyrollcall.com/everyone
39193,Poly Pocket,404,Flathead valley roller derby,United States,"6th December, 2022",http://www.derbyrollcall.com/everyone
39194,Roll Call,NaN,Chicago-Style Roller Derby,United States,"6th December, 2022",http://www.derbyrollcall.com/everyone
39195,Kaztastrophe,8,Illiana Derby Dames,United States,"6th December, 2022",http://www.derbyrollcall.com/everyone


In [26]:
# Get RDNation sitemap
rdn_sitemap_url = "https://rdnation.com/sitemap.xml"
rdn_sitemaps = adv.sitemap_to_df(rdn_sitemap_url)
# League pages have a specific URL structure
rdn_sitemaps["is_league"] = (
    rdn_sitemaps["loc"].str.contains("roller-derby-league/").fillna(False)
)


2022-12-06 13:27:04,225 | WARNING | sitemaps.py:530 | sitemap_to_df | reference to invalid character number: line 128356, column 73 https://rdnation.com/sitemaps?p=1
2022-12-06 13:27:04,759 | INFO | sitemaps.py:536 | sitemap_to_df | Getting https://rdnation.com/sitemaps?p=0


In [27]:
# Extract league URLs
rdn_league_urls = sorted(
    rdn_sitemaps[
        rdn_sitemaps["is_league"]
        & (
            rdn_sitemaps["loc"].str.contains("/2/")
            | rdn_sitemaps["loc"].str.contains("/1/")
        )
    ]["loc"].tolist()
)



In [28]:
# Loop through league pages and extract derby names
rdn_df = pd.DataFrame()
pbar = tqdm(rdn_league_urls)

for url in pbar:
    try:
        pbar.set_description(f"{len(rdn_df)} names found")
        league_df = pd.concat(pd.read_html(url), ignore_index=True)
        league_df.rename(columns={"Derby Name": "Name"}, inplace=True)
        league_df["url"] = url
        rdn_df = pd.concat([rdn_df, league_df])
    except ValueError as e:
        continue


37545 names found: 100%|██████████| 1389/1389 [06:12<00:00,  3.72it/s]


In [29]:
# If no data was found, load from CSV
if len(rdn_df) == 0:
    rdn_df = pd.read_csv("rdnation.csv")
else:
    rdn_df.dropna(how="all", axis="columns", inplace=True)
    rdn_df.drop_duplicates(subset=['Name','Number'], inplace=True)
    rdn_df.sort_values(by="Name", inplace=True)
    rdn_df.to_csv("rdnation.csv", index=False)
rdn_df


,Name,Number,url
5,!Wolfespit,16,https://rdnation.com/roller-derby-league/2/chi...
1,$3 bill,STAFF,https://rdnation.com/roller-derby-league/1/car...
0,'Lil Diablo,72,https://rdnation.com/roller-derby-league/1/ham...
1,.357 Maggie,6040,https://rdnation.com/roller-derby-league/1/cha...
11,.38 SpeShell,38,https://rdnation.com/roller-derby-league/1/lav...
...,...,...,...
14,Zulu Mother Smother,360,https://rdnation.com/roller-derby-league/1/m-i...
10,Zulu Xray,911,https://rdnation.com/roller-derby-league/1/rol...
88,Zuul,REF,https://rdnation.com/roller-derby-league/1/ora...
1,Zyklon C,HCN1,https://rdnation.com/roller-derby-league/2/cro...


In [30]:
# Fetch all names beginning with a letter from RollerDerbyRoster
def get_page_names(initial_letter, timeout=30):
    temp_names = []
    url = "https://rollerderbyroster.com/view-names/?ini={}".format(letter)
    # print("Downloading names from {}".format(url))
    try:
        response = session.get(url=url, timeout=timeout)
        r = session.get(url)
        soup = BeautifulSoup(r.text, "lxml")
        rows = soup.find_all("ul")
        # Use only last unordered list - this is where names are!
        for idx, li in enumerate(rows[-1]):
            # Name should be the text of the link within the list item
            name = li.find("a").get_text()
            temp_names.append(name)
    except requests.Timeout:
        print("Timeout!")
        pass
    return temp_names


In [31]:
# Get all names from RollerDerbyRoster
initial_letters = string.ascii_letters + string.digits + string.punctuation
rdr_df = pd.DataFrame()

pbar = tqdm(initial_letters)
for letter in pbar:
    try:
        pbar.set_description(f"{len(rdr_df)} names found - Processing {letter}")
        temp_names = get_page_names(initial_letter=letter)
        temp_df = pd.DataFrame(data={"Name": temp_names, "url": url})
        rdr_df = pd.concat([rdr_df, temp_df])
    except Exception as e:
        print(e)
        pass


82613 names found - Processing &:  71%|███████▏  | 67/94 [06:22<04:37, 10.29s/it]

Timeout!


82646 names found - Processing `:  95%|█████████▍| 89/94 [07:46<00:54, 10.92s/it]

Timeout!


82646 names found - Processing ~: 100%|██████████| 94/94 [07:59<00:00,  5.10s/it]


In [32]:
# If no data was found, load from CSV
if len(rdr_df) == 0:
    rdr_df = rdn_df.sort_values(by="Name").drop_duplicates(subset=['Name'])
    rdr_df = pd.read_csv("rollerderbyroster.csv")
else:
    rdr_df.to_csv("rollerderbyroster.csv", index=False)
rdr_df


,Name,url
0,A B Old Tricksee,https://rdnation.com/roller-derby-league/2/yuk...
1,A Blonde with No Name,https://rdnation.com/roller-derby-league/2/yuk...
2,A Bomb,https://rdnation.com/roller-derby-league/2/yuk...
3,A Bout To Snap,https://rdnation.com/roller-derby-league/2/yuk...
4,A Boy Named Rob,https://rdnation.com/roller-derby-league/2/yuk...
...,...,...
3,.50 Cal,https://rdnation.com/roller-derby-league/2/yuk...
4,.50 Caliber Killer,https://rdnation.com/roller-derby-league/2/yuk...
5,.50 Kal Kitten,https://rdnation.com/roller-derby-league/2/yuk...
6,.CC Rip Jaw,https://rdnation.com/roller-derby-league/2/yuk...


In [33]:
name_df = pd.concat([twoevils_df, drc_df, rdr_df, wftda_df, rdn_df], ignore_index=True)
# remove parenthetical phrases from names - eg "(cleared)"
name_df["Name"] = name_df["Name"].str.replace(r"\([^()]*\)", "").str.strip()
# remove names with only one character
name_df = name_df.loc[name_df["Name"].str.len() > 1]
name_df


C:\Users\dunn0172\AppData\Local\Temp\ipykernel_6560\2792732785.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  name_df["Name"] = name_df["Name"].str.replace(r"\([^()]*\)", "").str.strip()


,Name,Number,Registered,League,url,Country
0,!(ED,REF,2012-02-20,Santiago Roller Derby,https://www.twoevils.org/rollergirls/,NaN
1,!Wolfespit,16,2011-02-01,Chilli Padi Derby Grrrls,https://www.twoevils.org/rollergirls/,NaN
2,"""A"" Cup Annihilator",36A,2010-10-13,Billings Roller Derby,https://www.twoevils.org/rollergirls/,NaN
3,"""Bobby"" Val Halen",1984,2011-02-24,Roller Derby Quebec,https://www.twoevils.org/rollergirls/,NaN
4,"""Chupa'clark'bra""",333,2012-01-09,Killa Hurtz Roller Girls,https://www.twoevils.org/rollergirls/,NaN
...,...,...,...,...,...,...
188943,Zulu Mother Smother,360,NaN,NaN,https://rdnation.com/roller-derby-league/1/m-i...,NaN
188944,Zulu Xray,911,NaN,NaN,https://rdnation.com/roller-derby-league/1/rol...,NaN
188945,Zuul,REF,NaN,NaN,https://rdnation.com/roller-derby-league/1/ora...,NaN
188946,Zyklon C,HCN1,NaN,NaN,https://rdnation.com/roller-derby-league/2/cro...,NaN


In [34]:
name_df["Number"].value_counts()

REF        2719
13         1707
11         1081
7          1021
22          972
           ... 
sec 404       1
3p1c          1
10ty          1
3kHz          1
128ï¿½        1
Name: Number, Length: 11669, dtype: int64

In [ ]:
csv_file = Path("derby_names.csv")
name_df.drop_duplicates().sort_values(by=["Name"]).to_csv(csv_file, index=False)

if "google.colab" in sys.modules:
    files.download(csv_file)


In [ ]:
names_only = name_df[["Name"]].drop_duplicates().sort_values(by=["Name"])
names_only.to_csv("derby_names.txt", index=False, header=False)

if "google.colab" in sys.modules:
    files.download("derby_names.txt")


In [ ]:
numbers = name_df[["Number"]].copy()
numbers['Number'] = numbers['Number'].astype(str)
numbers = numbers.sort_values(by=['Number']).drop_duplicates()
numbers.to_csv("derby_numbers.txt", index=False, header=False)

if "google.colab" in sys.modules:
    files.download("derby_numbers.txt")


In [ ]:
names_numbers = (
    name_df[~name_df["Number"].isna()][["Name", "Number"]]
    .drop_duplicates()
    .sort_values(by=["Name", "Number"])
)
names_numbers.to_csv("derby_names_numbers.tsv", index=False, header=False, sep="\t")

if "google.colab" in sys.modules:
    files.download("derby_names_numbers.tsv")
